## Goal:
---
Working for the OilyGiant mining company, they need help finding the best place for a new well.

### Stages:
1. Data Preprocessing: Clean and organize the data, ensuring it's ready for analysis.
2. Exploratory Data Analysis (EDA): Perform an initial analysis to understand the data distribution and identify key trends.
3. ID Target/Feature Variables: Find out exactly what variable is being tested.
4. Split Dataset: Split dataset up into training, validating & test sets.
5. Train Regression Model: Train & evaluate the model for the task.
6. Calculate Profit: Find which region will generate the highest profit while minimizing losses.

## Import Libraries & Modules
---

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

## Load in Datasets
---

## Conduct Exploratory Data Analysis (EDA) / Null & Duplicates Check
---

In [2]:
region_0 = pd.read_csv('/datasets/geo_data_0.csv')
region_1 = pd.read_csv('/datasets/geo_data_1.csv')
region_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
def analyze_dataframe(df): 
    # Data types and non-null counts
    print("Column data types & number of non-null count")
    print("------------------------------")
    print(df.info())
    print("\n")
    
    # Display the first 5 rows of the DataFrame 
    print("First 5 rows of the DataFrame:") 
    print("------------------------------")
    print(df.head()) 
    print("\n")
    
    # Count the number of null values in each column
    num_nulls = df.isna().sum() 
    print("Number of null values in each column:")
    print("------------------------------")
    print(num_nulls) 
    print("\n")
    
    # Count the number of duplicate rows
    num_duplicates = df.duplicated().sum() 
    print("Number of duplicate rows:")
    print("------------------------------")
    print(num_duplicates)

In [4]:
analyze_dataframe(region_0)

Column data types & number of non-null count
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


First 5 rows of the DataFrame:
------------------------------
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


Number of null values in each column:
------------------------------
id         0
f0         0
f1         0
f2   

In [5]:
analyze_dataframe(region_1)

Column data types & number of non-null count
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


First 5 rows of the DataFrame:
------------------------------
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


Number of null values in each column:
------------------------------
id         0
f0         0
f1    

In [6]:
analyze_dataframe(region_2)

Column data types & number of non-null count
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


First 5 rows of the DataFrame:
------------------------------
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


Number of null values in each column:
------------------------------
id         0
f0         0
f1         0
f2   

After looking at the general layout of the dataframe for each region, it appears that there are no null or duplicate values. However, in the interest of creating accurate results with the model, the 'id' column should be dropped because it doesn't directly contribute to the prediction of volume of reserves in the oil wells.

## Drop 'id' column
---

In [7]:
region_0 = region_0.drop(columns=['id'])  # Drop specific columns
region_1 = region_1.drop(columns=['id'])  # Drop specific columns
region_2 = region_2.drop(columns=['id'])  # Drop specific columns

In [8]:
print(region_0.info())
print(region_1.info())
print(region_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float6

After dropping the 'id' column all of the regions, the datasets' features need to be scaled to ensure everything is on common ground.

## Train and test the model for each region

### Split the data into a training set and validation set at a ratio of 75:25
---

In [9]:
# Separates the original dataset into features/target, then scale numeric features, then split features & target into training & validation
def split_dataset(df, target_column, test_size=0.25, random_state=12345): 
    
    # Separate features and target 
    X = df.drop(target_column, axis=1) 
    y = df[target_column]
    
    # Scale Numeric Features
    scaler = StandardScaler() # Fit the scaler on the features and transform
    # Fit the scaler on the features and transform
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    # Split the dataset into training and validation sets 
    X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=test_size, random_state=random_state) 
    return X_train, X_valid, y_train, y_valid

In [10]:
X_train_0, X_valid_0, y_train_0, y_valid_0 = split_dataset(region_0, 'product', test_size=0.25, random_state=12345)

In [11]:
X_train_1, X_valid_1, y_train_1, y_valid_1 = split_dataset(region_1, 'product', test_size=0.25, random_state=12345)

In [12]:
X_train_2, X_valid_2, y_train_2, y_valid_2 = split_dataset(region_2, 'product', test_size=0.25, random_state=12345)

### Train the model and make predictions for the validation set
---

In [13]:
def linear_regression(X_train, y_train, X_valid, y_valid, state=12345):
       
        # Train the linear regression model on the bootstrap sample
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the resampled data 
        y_pred = model.predict(X_valid) 
        
        # Calculate the mean squared error for the bootstrap sample 
        mse = mean_squared_error(y_valid, y_pred)
        rmse = mse ** 0.5
        r2 = r2_score(y_valid, y_pred)

        # Store the coefficients and intercept
        coeffs = model.coef_
        intercept = model.intercept_
        
        # Predicting on the validation set using the mean coefficients and intercept
        pred = intercept + np.dot(X_valid, coeffs)
        avg_pred = np.mean(pred)


        return rmse, pred, avg_pred

### Save the predictions and correct answers for the validation set.
---

In [14]:
def convert(actual, predict):
    actual = actual.reset_index(drop=True)
    predict = pd.Series(predict)
    return actual, predict

In [15]:
rmse_0, preds_reg_0, avg_pred_0 = linear_regression(X_train_0, y_train_0, X_valid_0, y_valid_0)
acts_reg_0 = y_valid_0
acts_reg_0, preds_reg_0 = convert(acts_reg_0, preds_reg_0)

In [28]:
rmse_1, preds_reg_1, avg_pred_1 = linear_regression(X_train_1, y_train_1, X_valid_1, y_valid_1)
acts_reg_1 = y_valid_1
acts_reg_1, preds_reg_1 = convert(acts_reg_1, preds_reg_1)

In [29]:
rmse_2, preds_reg_2, avg_pred_2 = linear_regression(X_train_2, y_train_2, X_valid_2, y_valid_2)
acts_reg_2 = y_valid_2
acts_reg_2, preds_reg_2 = convert(acts_reg_2, preds_reg_2)

### Print the average volume of predicted reserves and model RMSE.
---

In [18]:
print('Region 0')
print('----------')
print("Average Volume of Predicted Reserves :", avg_pred_0)
print("RMSE:", rmse_0)
print('Region 1')
print('----------')
print("Average Volume of Predicted Reserves :", avg_pred_1)
print("RMSE:", rmse_1)
print('Region 2')
print('----------')
print("Average Volume of Predicted Reserves :", avg_pred_2)
print("RMSE:", rmse_2)

Region 0
----------
Average Volume of Predicted Reserves : 92.59256778438035
RMSE: 37.5794217150813
Region 1
----------
Average Volume of Predicted Reserves : 68.72854689544602
RMSE: 0.8930992867756171
Region 2
----------
Average Volume of Predicted Reserves : 94.96504596800489
RMSE: 40.02970873393434


In terms of average volumn of predicted reserves, region 1 seems to have the lowest average, while region 0 and 2 share similar averages. However, the RMSE for region 1 is the lowest indicating better model performance.

## Prepare for profit calculation
---

### Calculate the volume of reserves sufficient for developing a new well without losses

In [19]:
# The budget for development of 200 oil wells is 100 USD million; with 100,000,000 USD, 200 wells need to be developed
# 1 oil well = $500,000
# 1 Barrel = $4.50
# 1 Volume of Reserve/1000 Barrels = $4,500 ($4.50 * X = $4,500 | 4,500 / 4.5 = x | 1000 = x)
# How much volume is needed to justify building a new well without losses?

unit_profit = 4500
well_needs = 200
budget = 100000000
budget_per_well = 500000
for i in range(1, 1000):
    gross_volume = 4500 * i 
    if gross_volume >= budget_per_well:
        volume_min = i
        break
print("Minimum volume required to break at least even for budget per well:", volume_min)

Minimum volume required to break at least even for budget per well: 112


### Compare the obtained value with the average volume of reserves in each region

In [20]:
# Store Average Volume of Reserves
acts_avg_reg_0 = region_0['product'].mean()
acts_avg_reg_1 = region_1['product'].mean()
acts_avg_reg_2 = region_2['product'].mean()

In [21]:
# Compare the obtained value with the average volume of reserves in each region.
print('Minimum volume needed for a well:', volume_min)
print('Average volume of a well in region 0:', acts_avg_reg_0)
print('Average volume of a well in region 1:', acts_avg_reg_1)
print('Average volume of a well in region 2:', acts_avg_reg_2)

Minimum volume needed for a well: 112
Average volume of a well in region 0: 92.50000000000001
Average volume of a well in region 1: 68.82500000000002
Average volume of a well in region 2: 95.00000000000004


With the profit calculation step in mind, the two major variables prepared for that calculation are the ones that hold both the actual and predicted region values.

## Write a function to calculate profit from a set of selected oil wells and model predictions
---

### Pick the wells with the highest values of predictions. 

In [50]:
# Pick the wells with the highest values of predictions (200).
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 4500 * selected.sum()

In [70]:
state = np.random.RandomState(12345)

def calc_profit(act_reg, preds_reg):
    values = []
    for i in range(1000):
        target_subsample = act_reg.sample(n=500, replace=True, random_state=state)
        probs_subsample = preds_reg[target_subsample.index]

        values.append(revenue(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    mean = values.mean()
    
    # Calculate risk of loss (percentage of negative profit scenarios)
    risk_of_loss = (values < ((volume_min * 4500) * 200)).mean() * 100

    return mean, lower, upper, risk_of_loss

### Calculate risks and profit for each region

In [76]:
mean_0, lower_0, upper_0, risk_of_loss_0 = calc_profit(acts_reg_0, preds_reg_0)
print("Average revenue:", mean_0)
print("Lower Bound/2.5% Quantile:", lower_0)
print("Upper Bound/97.5% Quantile:", upper_0)
print("Risk of Loss:", risk_of_loss_0)

Average revenue: 104047087.12349911
Lower Bound/2.5% Quantile: 98751192.14501211
Upper Bound/97.5% Quantile: 109457642.66292138
Risk of Loss: 10.0


In [90]:
mean_1, lower_1, upper_1, risk_1 = calc_profit(acts_reg_1, preds_reg_1)
print("Average revenue:", mean_1)
print("Lower Bound/2.5% Quantile:", lower_1)
print("Upper Bound/97.5% Quantile:", upper_1)
print("Risk of Loss:", risk_1)

Average revenue: 105177860.28567389
Lower Bound/2.5% Quantile: 101237972.33037522
Upper Bound/97.5% Quantile: 109493927.23387702
Risk of Loss: 1.3


In [88]:
mean_2, lower_2, upper_2, risk_2 = calc_profit(acts_reg_2, preds_reg_2)
print("Average revenue:", mean_2)
print("Lower Bound/2.5% Quantile:", lower_2)
print("Upper Bound/97.5% Quantile:", upper_2)
print("Risk of Loss:", risk_2)

Average revenue: 104146260.17168252
Lower Bound/2.5% Quantile: 98148317.94529475
Upper Bound/97.5% Quantile: 109860543.07151984
Risk of Loss: 12.8


## Conclusion
---

### suggest a region for development of oil wells and justify the choice.
---

Based on the above calculations & the business conditions placed in the project, it seems that region 1 would be the best region for development of oil wells. This conclusion is based on the fact that this region holds the lowest predicted relative percentage for risk of loss and meets the requirement of being lower than 2.5%. Other than that, all regions seem to hold similar predicted average revenue but differ when it comes to the chances of loss.